# Analysis of MD Trajectory
**Molecular Dynamics (MD) Simulation**: A computational method that simulates the physical movement of atoms and molecules over time by solving Newton's equations of motion.

**Applications**: Used to study protein folding, ligand binding, molecular interactions, and other biochemical processes at the atomic level.

**Trajectory Analysis**: Involves extracting information such as:

*   RMSD (Root Mean Square Deviation) to assess the stability of molecular structures.
*   RMSF (Root Mean Square Fluctuation) to evaluate flexibility of different regions of a molecule.
*   RDF (Radial Distribution Function) to study the spatial arrangement of atoms in the system.
*   Principal Component Analysis (PCA): A dimensionality reduction technique used to identify dominant motion modes in the system, simplifying the interpretation of complex data.

https://www.mdanalysis.org/




https://github.com/avirshup/py3dmol


In [ ]:
!pip install MDAnalysis  nglview py3dmol numpy scikit-learn matplotlib

In [ ]:
#import tools
# MDAnalysis tools
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align, pca
# To view the trajs on notebook
import nglview as nv

In [ ]:
# Data processing
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)  #Suppress scientific notation

from sklearn.decomposition import PCA

In [ ]:
# Plot & System tools
import matplotlib as mpl
import matplotlib.pyplot as plt
from  matplotlib.colors import ListedColormap, NoNorm, BoundaryNorm, CSS4_COLORS
import seaborn as sns

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

#Change working directory to current directory
import os
os.chdir(os.getcwd() )

In [ ]:
# We will download the sample trajectory data from https://github.com/Rajnishphe

#https://www.rcsb.org/structure/6SAF

!wget https://raw.githubusercontent.com/Rajnishphe/MolecularDynamics-Tutorial/main/Trajectory_files/md.gro
!wget https://raw.githubusercontent.com/Rajnishphe/MolecularDynamics-Tutorial/main/Trajectory_files/md_center.xtc


In [ ]:
ls

In [ ]:
u = mda.Universe("md.gro","md_center.xtc")
print (u)
print (u.atoms)

In [ ]:
print('Atoms: ', u.atoms.n_atoms)
print('Residues: ', u.residues.n_residues)
print('Segments: ', u.segments.n_segments)

In [ ]:
import os
import py3Dmol

# --- Helper classes ---
class Atom(dict):
    def __init__(self, line):
        self["type"] = line[0:6].strip()
        self["idx"] = line[6:11].strip()
        self["name"] = line[12:16].strip()
        self["resname"] = line[17:20].strip()
        self["resid"] = int(line[22:26])
        self["x"] = float(line[30:38])
        self["y"] = float(line[38:46])
        self["z"] = float(line[46:54])
        self["sym"] = line[76:78].strip() if len(line) >= 78 else self["name"][0].upper()

    def __str__(self):
        line = list(" " * 80)
        line[0:6] = self["type"].ljust(6)
        line[6:11] = self["idx"].rjust(5)
        line[12:16] = self["name"].ljust(4)
        line[17:20] = self["resname"].rjust(3)
        line[22:26] = str(self["resid"]).rjust(4)
        line[30:38] = f"{self['x']:8.3f}"
        line[38:46] = f"{self['y']:8.3f}"
        line[46:54] = f"{self['z']:8.3f}"
        line[76:78] = self["sym"].rjust(2)
        return "".join(line) + "\n"

class Molecule(list):
    def __init__(self, file):
        for line in file:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                self.append(Atom(line))

# --- Use the already loaded universe `u` ---
protein = u.select_atoms("protein")
stride_animation = 100
models = ""

os.makedirs("frames", exist_ok=True)

for i, ts in enumerate(u.trajectory[::stride_animation]):
    temp_pdb = f"frames/frame_{i}.pdb"
    with mda.Writer(temp_pdb, protein.n_atoms) as W:
        W.write(protein)
    with open(temp_pdb) as f:
        mol = Molecule(f)
        models += f"MODEL     {i+1}\n"
        for atom in mol:
            models += str(atom)
        models += "ENDMDL\n"

# --- Visualize with py3Dmol ---
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(models)
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.animate({'loop': 'forward'})
view.show()


In [ ]:
from MDAnalysis.analysis import align, rms
aligner = align.AlignTraj(u, u, select='name CA', in_memory=True).run()

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
#make protein, ligand and complex as independent entries
protein = u.atoms.select_atoms("protein")
print (protein)

In [ ]:
import os
import py3Dmol

# --- Helper classes ---
class Atom(dict):
    def __init__(self, line):
        self["type"] = line[0:6].strip()
        self["idx"] = line[6:11].strip()
        self["name"] = line[12:16].strip()
        self["resname"] = line[17:20].strip()
        self["resid"] = int(line[22:26])
        self["x"] = float(line[30:38])
        self["y"] = float(line[38:46])
        self["z"] = float(line[46:54])
        self["sym"] = line[76:78].strip() if len(line) >= 78 else self["name"][0].upper()

    def __str__(self):
        line = list(" " * 80)
        line[0:6] = self["type"].ljust(6)
        line[6:11] = self["idx"].rjust(5)
        line[12:16] = self["name"].ljust(4)
        line[17:20] = self["resname"].rjust(3)
        line[22:26] = str(self["resid"]).rjust(4)
        line[30:38] = f"{self['x']:8.3f}"
        line[38:46] = f"{self['y']:8.3f}"
        line[46:54] = f"{self['z']:8.3f}"
        line[76:78] = self["sym"].rjust(2)
        return "".join(line) + "\n"

class Molecule(list):
    def __init__(self, file):
        for line in file:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                self.append(Atom(line))

# --- Use the already loaded universe `u` ---
protein = u.select_atoms("protein")
stride_animation = 100
models = ""

os.makedirs("frames", exist_ok=True)

for i, ts in enumerate(u.trajectory[::stride_animation]):
    temp_pdb = f"frames/frame_{i}.pdb"
    with mda.Writer(temp_pdb, protein.n_atoms) as W:
        W.write(protein)
    with open(temp_pdb) as f:
        mol = Molecule(f)
        models += f"MODEL     {i+1}\n"
        for atom in mol:
            models += str(atom)
        models += "ENDMDL\n"

# --- Visualize with py3Dmol ---
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(models)
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.animate({'loop': 'forward'})
view.show()


#Protein RMSD
RMSD (Root Mean Square Deviation) is a quantitative measure of the average distance between atoms (typically backbone or Cα atoms) of protein structures over time.
It reflects how much a protein conformation changes during a molecular dynamics (MD) simulation compared to a reference structure (usually the initial frame).

**Importance**

*   Monitors Structural Stability: A plateau in RMSD indicates the protein has reached a stable conformation.
*   Detects Conformational Changes: Helps in identifying whether a protein undergoes significant structural shifts (e.g., folding, unfolding, domain movements).
*   Assesses Equilibration: Used to verify if the system is equilibrated before analyzing further simulation data.
*   Compares Different Conditions: Useful for comparing structural dynamics across mutants, liganded vs. unliganded forms, or different force fields.
*   Supports Drug Design: In ligand-bound simulations, RMSD reveals if the binding pocket or ligand remains stable.



In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import rms
import matplotlib.pyplot as plt

R = rms.RMSD(protein, protein, select="backbone", ref_frame=1) # backbone (typically N, CA, C, O), name CA, protein, resid 50-100
R.run()

rmsd = R.rmsd.T  # Transpose for easier plotting
time = rmsd[1]/1000
values = rmsd[2]

# Plotting
fig, ax = plt.subplots(figsize=(8, 4))
#ax.plot(time, values, 'k-', label="backbone")
ax.plot(time, values, color='teal', linewidth=2, label="backbone")

ax.set_xlabel("Time (ns)")
ax.set_ylabel(r"RMSD ($\AA$)")
ax.legend(loc="best")
fig.tight_layout()
fig.savefig("rmsd_backbone.tif")


#Protein RMSF Calculation

RMSF (Root Mean Square Fluctuation) quantifies the average positional deviation of each atom or residue from its mean position during a molecular dynamics (MD) simulation.

It provides residue-level insight into how flexible different parts of a protein are over time.

 **Importance**

*   Highlights Flexible and Rigid Regions: Helps identify loop regions, terminal ends, or domains with high mobility versus stable core regions.
*   Assesses Binding Effects: Reveals how ligand binding or mutation affects protein flexibility at specific sites.
*   Supports Functional Insights: Flexible regions may correspond to active sites, allosteric sites, or interaction interfaces.
*   Validates Simulations: Helps compare simulated flexibility patterns with experimental B-factors (from X-ray crystallography).
*   Enables Drug Design: Detects flexible pockets or hotspots for ligand binding.

In [ ]:
c_alphas = u.select_atoms('protein and name CA')
R = rms.RMSF(c_alphas).run()
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

fig, ax = plt.subplots(figsize=(8, 4))

# Plot residue-wise RMSF (example using C-alpha atoms)
ax.plot(c_alphas.resids, R.rmsf, color='teal', linewidth=2, label="C-alpha")

ax.set_xlabel("Residue Number")
ax.set_ylabel(r"RMSF ($\AA$)")
ax.legend(loc="best")

fig.tight_layout()
fig.savefig("rmsf_c_alpha.tif")


# Save the RMSF values as custom B factors and display the 3D structure



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import py3Dmol
import MDAnalysis as mda

# Step 1: Create DataFrame with RMSF values
rmsf_data = pd.DataFrame(R.rmsf, index=c_alphas.resnums, columns=['RMSF'])

# Step 2: Save protein structure with RMSF as B-factors
protein = u.select_atoms("protein")
protein.write("protein_tmp.pdb")

u_rmsf = mda.Universe("protein_tmp.pdb")
u_rmsf.add_TopologyAttr("tempfactors")

# Step 3: Map RMSF to B-factors for all atoms using residue IDs
rmsf_map = rmsf_data['RMSF'].to_dict()
rmsf_3d = [rmsf_map.get(atom.resid, 0.0) for atom in u_rmsf.atoms]
u_rmsf.atoms.tempfactors = rmsf_3d

# Save new PDB with custom B-factors
u_rmsf.atoms.write("protein_rmsf.pdb")

# Step 4: Visualize using py3Dmol
def show_rmsf(pdb_file, min_rmsf, max_rmsf):
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    with open(pdb_file) as f:
        view.addModel(f.read(), 'pdb')
    view.setStyle({'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': min_rmsf, 'max': max_rmsf}}})
    view.zoomTo()
    return view

def plot_colorbar(min_rmsf, max_rmsf):
    fig, ax = plt.subplots(figsize=(6, 1))
    fig.subplots_adjust(bottom=0.5)
    cmap = mpl.cm.jet_r
    norm = mpl.colors.Normalize(vmin=min_rmsf, vmax=max_rmsf)
    cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
    cb1.set_label('RMSF (Å)', fontsize=14)
    cb1.ax.tick_params(labelsize=12)
    plt.show()

# Show RMSF 3D map and colorbar
min_rmsf = float(np.min(R.rmsf))
max_rmsf = float(np.max(R.rmsf))

show_rmsf("protein_rmsf.pdb", min_rmsf, max_rmsf).show()
plot_colorbar(min_rmsf, max_rmsf)



# Radius of Gyration (Rg)
The radius of gyration (Rg) is a measure of the compactness or overall size of a protein structure. It quantifies the distribution of atomic positions around the center of mass of the protein. In simple terms, it describes how "spread out" or "compact" a protein's structure is at any given time.

It is often calculated using the atoms of the protein (usually Cα atoms) and is used to assess changes in the protein's conformation during a molecular dynamics (MD) simulation.


**Importance**
*   Monitors Protein Folding: As a protein folds, its Rg typically decreases as the structure becomes more compact. A larger Rg usually indicates a more extended or unfolded state.

*   Identifies Structural Transitions: Sudden changes in Rg can indicate major conformational changes in the protein, such as folding, unfolding, or domain movements.

*   Checks Protein Stability: A plateau in Rg over time signifies that the protein has reached a stable state and is no longer undergoing significant conformational changes.

*   Assesses Solvent Accessibility: Larger values of Rg can indicate more exposed, solvent-accessible areas in the protein structure.

*   Evaluates Different States of Proteins: Rg is useful in comparing different protein states, such as folded versus unfolded states, or bound versus unbound states.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Select CA atoms
c_alpha = u.select_atoms("name CA")
print(f"Selected {len(c_alpha)} CA atoms")

# Calculate radius of gyration per frame and get time
rg_list = []
time_list = []

for ts in u.trajectory:
    rg = c_alpha.radius_of_gyration()
    rg_list.append(rg)
    time_list.append(u.trajectory.time / 1000.0)  # ps to ns

# Convert to arrays
rg_array = np.array(rg_list)
time_array = np.array(time_list)

# Plotting
plt.figure(figsize=(8, 4))
plt.plot(time_array, rg_array, alpha=0.6, color='green', linewidth=1.0)
plt.xlabel("Time (ns)", fontsize=14, fontweight='bold')
plt.ylabel("Radius of gyration ($\AA$)", fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title("Radius of Gyration vs Time", fontsize=14)
plt.tight_layout()
plt.show()

plt.savefig("RoG.tif")

#Pairwise RMSD calculation

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import diffusionmap
import matplotlib.pyplot as plt

# Compute distance matrix using alpha carbons (CA atoms)
matrix = diffusionmap.DistanceMatrix(u, select='name CA').run()

# Plot the distance matrix
plt.figure(figsize=(6, 5))
plt.imshow(matrix.dist_matrix, cmap='viridis', aspect='auto')
plt.xlabel('Frame')
plt.ylabel('Frame')
plt.colorbar(label=r'RMSD ($\AA$)')
plt.title('Diffusion Map Distance Matrix')
plt.tight_layout()
plt.show()


# Principal Component Analysis
Principal Component Analysis (PCA) is a dimensionality reduction technique used to simplify complex datasets by identifying patterns and highlighting the most significant variations in the data.

In [ ]:
import MDAnalysis as mda
import MDAnalysis.analysis.pca as pca
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

# PCA analysis on the backbone atoms
backbone = u.select_atoms('backbone')
n_bb = len(backbone)

# Perform PCA on the backbone atoms
pc = pca.PCA(u, select='backbone')
pc.run()

# Variance and Cumulative Variance
print(pc.variance[0])  # Variance of the first PC
print(pc.cumulated_variance[0])  # Cumulative variance of the first PC
print(pc.cumulated_variance[5])  # Cumulative variance of the 5th PC

# Plot Cumulative Variance
plt.plot(pc.cumulated_variance[:50])
plt.xlabel('Principal component')
plt.ylabel('Cumulative variance')
plt.title('Cumulative Variance Explained by Principal Components')
plt.show()

# Transform the data using the first 5 principal components
transformed = pc.transform(backbone, n_components=5)
print(f"Shape of transformed data: {transformed.shape}")

# Create a DataFrame for the first 5 PCs and add time data
df = pd.DataFrame(transformed, columns=['PC{}'.format(i+1) for i in range(5)])
df['Time (ps)'] = df.index * u.trajectory.dt
df.head()

# Print the number of backbone atoms and principal components
print(f'There are {n_bb} backbone atoms in the analysis')
print(f'Principal Components Shape: {pc.p_components.shape}')




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import gaussian_kde
import os

# Assuming the 'df' DataFrame is already created with PCA data
# Provide which principal component you want to plot:
PC_x = "1"  # Replace with the desired PC (e.g., 1, 2, 3, ...)
PC_y = "3"  # Replace with the desired PC (e.g., 3, 4, 5, ...)

# Extracting data for the specified PCs
PC1 = df[f'PC{PC_x}'].tolist()
PC2 = df[f'PC{PC_y}'].tolist()

# Create the figure for the 2D plot
plt.figure(figsize=(5, 5))
plt.scatter(PC1, PC2, s=5, color="gray", alpha=0.2)

# Plot a KDE contour map (no fill)
sns.kdeplot(x=PC1, y=PC2, cmap="viridis", levels=10, linewidths=1.5)

plt.xlabel(f"PC{PC_x}", fontsize=14, fontweight='bold')
plt.ylabel(f"PC{PC_y}", fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Saving the 2D plot
workDir = './output'  # Replace with your actual working directory
os.makedirs(workDir, exist_ok=True)
plt.savefig(os.path.join(workDir, f"PC{PC_x}_PC{PC_y}_2D.png"), dpi=600, bbox_inches='tight')

# Compute KDE for the 3D plot
xy = np.vstack([PC1, PC2])
kde = gaussian_kde(xy)(xy)  # Evaluates the density at each point

# Create the 3D plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with Z as density
ax.scatter(PC1, PC2, kde, c=kde, cmap='viridis')

# Labels for 3D plot
ax.set_xlabel(f"PC{PC_x}", fontsize=14, fontweight='bold')
ax.set_ylabel(f"PC{PC_y}", fontsize=14, fontweight='bold')
ax.set_zlabel("Density (Z)", fontsize=11, fontweight='bold')
ax.set_title("3D Density Plot of PCA Data")

# Saving the 3D plot
plt.savefig(os.path.join(workDir, f"PC{PC_x}_PC{PC_y}_3D.png"), dpi=600, bbox_inches='tight')

# Save the principal component data to CSV files
pc1 = pd.DataFrame(PC1)
pc1.to_csv(os.path.join(workDir, f"PC{PC_x}.csv"))

pc2 = pd.DataFrame(PC2)
pc2.to_csv(os.path.join(workDir, f"PC{PC_y}.csv"))
